### 이전 코드에서 생성한 데이터 불러오기

In [2]:
import numpy as np
# continual learning을 위한 데이터셋

train_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
train_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_A.npy")
train_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_B.npy")
train_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_C.npy")

train_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_A.npy")
train_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_B.npy")
train_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_C.npy")

val_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
val_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_A.npy")
val_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_B.npy")
val_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_C.npy")

val_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_A.npy")
val_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_B.npy")
val_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_C.npy")

test_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
test_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_A.npy")
test_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_B.npy")
test_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_C.npy")

test_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_A.npy")
test_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_B.npy")
test_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_C.npy")


# joint training 을 위한 데이터셋

X_train_3 = np.load("../dataset/joint_training_dataset/X_joint_train_data.npy")
y_train_3 = np.load("../dataset/joint_training_dataset/y_joint_train_data.npy")
X_val_3 = np.load("../dataset/joint_training_dataset/X_joint_val_data.npy")
y_val_3 = np.load("../dataset/joint_training_dataset/y_joint_val_data.npy")
X_test_3 = np.load("../dataset/joint_training_dataset/X_joint_test_data.npy")
y_test_3 = np.load("../dataset/joint_training_dataset/y_joint_test_data.npy")


In [3]:
train_data_level3["B"][0][753]                 # 756,4096

array([-1.208034 ,  0.5018176,  0.1888077, ..., -0.9396821,  0.4002334,
       -0.5487765])

In [4]:
test_data_level3["C"][0]

array([[-0.00088406, -0.0018706 ,  0.00108903, ..., -0.00154175,
        -0.00351483, -0.00581676],
       [ 0.00470634,  0.00404865,  0.00306211, ...,  0.00898136,
         0.0099679 ,  0.00963905],
       [ 0.0043775 ,  0.00536404,  0.0043775 , ...,  0.00602173,
         0.00635058,  0.00536404],
       ...,
       [ 0.00273326,  0.00569289,  0.00799482, ...,  0.00273326,
         0.00141787,  0.00076018],
       [ 0.01029675,  0.00832366,  0.00766597, ...,  0.01818908,
         0.01522945,  0.01226983],
       [ 0.00470634,  0.00536404,  0.00733712, ...,  0.00043133,
         0.00207557,  0.00207557]])

### 데이터로더 래핑

In [5]:
# 상위 디렉토리로 작업환경 이동
import os
os.chdir(os.path.join(globals()['_dh'][0], ".."))

In [6]:
from dfb.dataset import *
from dfb.processing import *
import experiment
import torchvision.transforms as transforms

model_name = "wdcnn2"

sample_length = experiment.get_sample_length(model_name)
tf_data = experiment.get_transform(model_name)
tf_data = transforms.Compose(tf_data)
tf_label = NpToTensor()
batch_size = 128
num_worker = 4

data_handler = DatasetHandler()

data_handler.assign(
    X_train_3, y_train_3, X_val_3, y_val_3, X_test_3, y_test_3,
    sample_length, "3", tf_data, tf_label, batch_size, num_worker
)

### 모델 가져오기

In [7]:

# 모델 수정되는 경우 해당 셀에서 바로 갱신되도록 autoreload사용
%load_ext autoreload
%autoreload 2

from dfb.model.wdcnn2 import *
from dfb.model.wdcnn3 import *
from dfb.model.wdcnn4 import *
# 클래스 수는 맘대로 지정

# 클래스 수 10개짜리 모델 만들기
model = WDCNN2(n_classes=3)

### 훈련하기

In [8]:
from dfb.trainmodule import *

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

# ticnn 플래그는 일단 신경쓸 필요 X
training_module = PlModule(model, optimizer, loss, True)

n_steps_d = len(data_handler.dataloaders["3"]["train"].dataset) // (batch_size)

# val_loss 가 가장 작은 모델을 저장하는 콜백
callback = pl.callbacks.ModelCheckpoint(monitor="val_loss",
                                        dirpath=f"./logs/test/best_model",
                                        filename=f"model",
                                        save_top_k=1,
                                        mode="min")
trainer = pl.Trainer(
    accelerator="auto",
    #gpus = [0],
    max_epochs=100,
    val_check_interval= n_steps_d,
    default_root_dir="./logs/test",
    callbacks=[callback]
)

result = trainer.fit(model=training_module,
                     train_dataloaders=data_handler.dataloaders["3"]["train"],
                     val_dataloaders=data_handler.dataloaders["3"]["val"])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /data/home/happy113200/DeepFault/logs/test/best_model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### 테스트하기

In [59]:
training_module.load_from_checkpoint(f"./logs/test/best_model/model-v36.ckpt",
                                            model=model, optimizer=optimizer,
                                            loss_fn=loss)
result = trainer.test(model=training_module, dataloaders=data_handler.dataloaders["3"]["test"])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 106.00it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9853723645210266
        test_loss          0.054738495498895645
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [12]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from dfb.trainmodule import PlModule

In [13]:
aaa = training_module.load_from_checkpoint("./logs/test/best_model/model-v35.ckpt", optimizer, loss)

TypeError: 'Adam' object is not callable

In [10]:
import torch
from dfb.trainmodule import *  # 모듈 및 클래스 임포트

# 1. 훈련된 모델의 체크포인트 파일 경로
model_checkpoint_path = "./logs/test/best_model/model-v36.ckpt"  # 체크포인트 파일 경로

# 2. 모델 초기화 및 훈련된 가중치 불러오기
checkpoint = torch.load(model_checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])  # 훈련된 가중치 불러오기

RuntimeError: Error(s) in loading state_dict for WDCNN2:
	Missing key(s) in state_dict: "conv_layers.0.weight", "conv_layers.0.bias", "conv_layers.1.weight", "conv_layers.1.bias", "conv_layers.1.running_mean", "conv_layers.1.running_var", "conv_layers.4.weight", "conv_layers.4.bias", "conv_layers.5.weight", "conv_layers.5.bias", "conv_layers.5.running_mean", "conv_layers.5.running_var", "conv_layers.8.weight", "conv_layers.8.bias", "conv_layers.9.weight", "conv_layers.9.bias", "conv_layers.9.running_mean", "conv_layers.9.running_var", "conv_layers.12.weight", "conv_layers.12.bias", "conv_layers.13.weight", "conv_layers.13.bias", "conv_layers.13.running_mean", "conv_layers.13.running_var", "linear_layers.0.weight", "linear_layers.0.bias", "linear_layers.1.weight", "linear_layers.1.bias", "linear_layers.1.running_mean", "linear_layers.1.running_var", "head.weight", "head.bias". 
	Unexpected key(s) in state_dict: "model.conv_layers.0.weight", "model.conv_layers.0.bias", "model.conv_layers